In [1]:
import torch

## Carregar os Dados

In [2]:
train_data = []
test_data = []

for i in range(1,35):
    train_data.append(torch.load('separed_timestamps/without_unknowns/train/' + str(i) + '.pt'))

for i in range(35,50):
    test_data.append(torch.load('separed_timestamps/without_unknowns/test/' + str(i) + '.pt'))

In [3]:
train_data

[Data(adjacency_matrix=[2147, 2147], edge_index=[2, 1924], x=[2147, 166], y=[2147]),
 Data(adjacency_matrix=[1117, 1117], edge_index=[2, 858], x=[1117, 166], y=[1117]),
 Data(adjacency_matrix=[1279, 1279], edge_index=[2, 727], x=[1279, 166], y=[1279]),
 Data(adjacency_matrix=[1440, 1440], edge_index=[2, 1169], x=[1440, 166], y=[1440]),
 Data(adjacency_matrix=[1882, 1882], edge_index=[2, 1491], x=[1882, 166], y=[1882]),
 Data(adjacency_matrix=[485, 485], edge_index=[2, 209], x=[485, 166], y=[485]),
 Data(adjacency_matrix=[1203, 1203], edge_index=[2, 858], x=[1203, 166], y=[1203]),
 Data(adjacency_matrix=[1165, 1165], edge_index=[2, 1044], x=[1165, 166], y=[1165]),
 Data(adjacency_matrix=[778, 778], edge_index=[2, 484], x=[778, 166], y=[778]),
 Data(adjacency_matrix=[972, 972], edge_index=[2, 538], x=[972, 166], y=[972]),
 Data(adjacency_matrix=[696, 696], edge_index=[2, 477], x=[696, 166], y=[696]),
 Data(adjacency_matrix=[506, 506], edge_index=[2, 446], x=[506, 166], y=[506]),
 Data(ad

## Definir o Modelo

In [4]:
import torch.nn.functional as F
import torch_geometric.nn as nn

In [5]:
hidden_size = 100
n_classes = 2


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = nn.GCNConv(166,100, bias=False)
        self.act1 = torch.nn.ReLU()
        self.conv2 = nn.GCNConv(100,2, bias=False)
        self.act2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x, edge_index, batch_index):
        hidden1 = self.conv1(x, edge_index)
        hidden1 = self.act1(hidden1)
        hidden2 = self.conv2(hidden1, edge_index)
        output = self.act2(hidden2)
        
        return hidden2, output

In [6]:
model = GCN()

In [7]:
model

GCN(
  (conv1): GCNConv(166, 100)
  (act1): ReLU()
  (conv2): GCNConv(100, 2)
  (act2): Softmax(dim=1)
)

In [8]:
## Use a GPU para treinar
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
loss = torch.nn.CrossEntropyLoss(weight=torch.Tensor([0.7, 0.3]))

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for ts, data in enumerate(train_data):
        for epoch in range(1000):
            # Usar GPU
            data.to(device)
            # Resetar Gradientes
            optimizer.zero_grad()
            # Passando as informações do batch e de conexão dos grafos
            hidden, logits = model(data.x.float(), data.edge_index, None)
            #label_pred = pred.max(1)[1]
            # Calculando a perda e os gradientes.
            l = loss(logits, data.y)
            l.backward()
            # Atualizar usando os gradientes.
            optimizer.step()
        if (epoch + 1) % 100 == 0:
            print('ts',ts+1,'epoch =', epoch + 1, 'loss =', l.item())

In [12]:
train()

ts 1 epoch = 1000 loss = 0.3229762017726898
ts 2 epoch = 1000 loss = 0.31532296538352966
ts 3 epoch = 1000 loss = 0.3240889608860016
ts 4 epoch = 1000 loss = 0.32273465394973755
ts 5 epoch = 1000 loss = 0.3182002305984497
ts 6 epoch = 1000 loss = 0.318007230758667
ts 7 epoch = 1000 loss = 0.33772048354148865
ts 8 epoch = 1000 loss = 0.3486177623271942
ts 9 epoch = 1000 loss = 0.3419018089771271
ts 10 epoch = 1000 loss = 0.3273267447948456
ts 11 epoch = 1000 loss = 0.3335708677768707
ts 12 epoch = 1000 loss = 0.3303285539150238
ts 13 epoch = 1000 loss = 0.3492192327976227
ts 14 epoch = 1000 loss = 0.3399658799171448
ts 15 epoch = 1000 loss = 0.3304792642593384
ts 16 epoch = 1000 loss = 0.34990212321281433
ts 17 epoch = 1000 loss = 0.3496834933757782
ts 18 epoch = 1000 loss = 0.34453675150871277
ts 19 epoch = 1000 loss = 0.3590601682662964
ts 20 epoch = 1000 loss = 0.4026060998439789
ts 21 epoch = 1000 loss = 0.4032316207885742
ts 22 epoch = 1000 loss = 0.3497554659843445
ts 23 epoch = 1

In [13]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        for data in test_data:
            data.to(device)
            _, logits = model(data.x.float(), data.edge_index, None)
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [14]:
test()

In [15]:
label_pred_list

[1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [16]:
y_true_list

[1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,


In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [18]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [19]:
print(classification_report(y_true_list,label_pred_list))

              precision    recall  f1-score   support

           0       0.89      0.46      0.61      1083
           1       0.96      1.00      0.98     15587

    accuracy                           0.96     16670
   macro avg       0.92      0.73      0.79     16670
weighted avg       0.96      0.96      0.96     16670



In [20]:
prec

0.9639796298596448

In [21]:
rec

0.9958298582151793

In [22]:
f1

0.9796459339202879

In [23]:
# https://www.kaggle.com/karthikapv/gcn-elliptic-dataset Refazer com implementação manual.